# Ensemble: Gradient

In [39]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

Vamos a pobar usando Gradient Boosting y para ello vamos a usar las tres técnicas de características y de reducción de instancia que mejor resultado nos ha dado que son:
- mrMr con centroides hard
- RFC con centroides soft
- RFC con centroides hard

Se usa Friedman como criterio para medir la calidad de la particición.

In [40]:
datasets = {
    "reduce_mrmr_instances_hard": pd.read_csv('./data/df_reduce_mrmr_instances_hard.csv'),
    "reduce_RFC_instances": pd.read_csv('./data/df_reduce_RFC_instances.csv'),
    "reduce_RFC_instances_hard": pd.read_csv('./data/df_reduce_RFC_instances_hard.csv'),
    "test_data" : pd.read_csv('./data/test_data.csv')    
}

In [41]:
# Seleccionamos los datos de prueba del conjunto de datos
data_prueba = datasets['test_data']

# Separar las características (X) de la variable objetivo (y)
x = data_prueba.drop(columns=['Class'])
y_test_total = data_prueba['Class']

# Filtramos las columnas relevantes según mRMR
columns_to_keep = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']
x_reduced = x[columns_to_keep]

# Crear el scaler para normalizar los datos
scaler = MinMaxScaler()

# Normalizamos las columnas seleccionadas
x_reduced['Amount'] = scaler.fit_transform(x_reduced[['Amount']])
x_reduced['Time'] = scaler.fit_transform(x_reduced[['Time']])

C:\Users\joses\AppData\Local\Temp\ipykernel_14116\1107825650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_reduced['Amount'] = scaler.fit_transform(x_reduced[['Amount']])
C:\Users\joses\AppData\Local\Temp\ipykernel_14116\1107825650.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_reduced['Time'] = scaler.fit_transform(x_reduced[['Time']])


In [42]:
from sklearn.preprocessing import MinMaxScaler
# Cogemos los datos de test y les eliminamos las cracterísticas que no necesitamos
X = data_prueba.drop(columns=['Class'])
y_test_final = data_prueba['Class']
### Version con RandomForestClassifier
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']
X_reduce_RFC = X[columns_to_keep_RFC]

print(y_test_final.value_counts())

Class
0    28432
1       49
Name: count, dtype: int64


## Gradient Boosting con RFC clusterCentroids_hard

In [43]:
df_reduce_mrmr_instances = datasets["reduce_RFC_instances_hard"]
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

# Configura el modelo Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=0)

# Define el rango de hiperparámetros para GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'max_depth': [3, 4, 5],  # Profundidad máxima de los árboles
    'subsample': [0.8, 1.0],  # Fracción de muestras para entrenar cada árbol
    'max_features': ['sqrt', 'log2', None]  # Máximo de características consideradas
}

# Configura GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Evalúa el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros: {'learning_rate': 0.01, 'max_depth': 4, 'max_features': None, 'n_estimators': 100, 'subsample': 0.8}
Mejor puntuación de validación cruzada: 0.9336230146838478


## Ejecutamos para el conjunto de prueba

In [44]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_reduce_RFC)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[27734   698]
 [    6    43]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.98      0.99     28432
Fraudulentas       0.06      0.88      0.11        49

    accuracy                           0.98     28481
   macro avg       0.53      0.93      0.55     28481
weighted avg       1.00      0.98      0.99     28481



## Gradient Boosting con mrMr clusterCentroids_hard

In [45]:
df_reduce_mrmr_instances = datasets["reduce_mrmr_instances_hard"]
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

# Configura el modelo Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=0)

# Define el rango de hiperparámetros para GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'max_depth': [3, 4, 5],  # Profundidad máxima de los árboles
    'subsample': [0.8, 1.0],  # Fracción de muestras para entrenar cada árbol
    'max_features': ['sqrt', 'log2', None]  # Máximo de características consideradas
}

# Configura GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros: {'learning_rate': 0.2, 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200, 'subsample': 0.8}
Mejor puntuación de validación cruzada: 0.9237838377784436


## Ejecutamos para el conjunto de prueba

In [46]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_reduced)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[26620  1812]
 [    4    45]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.94      0.97     28432
Fraudulentas       0.02      0.92      0.05        49

    accuracy                           0.94     28481
   macro avg       0.51      0.93      0.51     28481
weighted avg       1.00      0.94      0.97     28481



## Gradient Boosting con RFC clusterCentroids_soft

In [47]:
df_reduce_RFC_instances = datasets["reduce_RFC_instances"]
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

# Configura el modelo Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=0)

# Define el rango de hiperparámetros para GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'max_depth': [3, 4, 5],  # Profundidad máxima de los árboles
    'subsample': [0.8, 1.0],  # Fracción de muestras para entrenar cada árbol
    'max_features': ['sqrt', 'log2', None]  # Máximo de características consideradas
}

# Configura GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros y su desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)


Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros: {'learning_rate': 0.01, 'max_depth': 3, 'max_features': None, 'n_estimators': 200, 'subsample': 0.8}
Mejor puntuación de validación cruzada: 0.9265408051143741


In [48]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_reduce_RFC)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[27852   580]
 [    7    42]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.98      0.99     28432
Fraudulentas       0.07      0.86      0.13        49

    accuracy                           0.98     28481
   macro avg       0.53      0.92      0.56     28481
weighted avg       1.00      0.98      0.99     28481



# XGBoost con mrmr instances hard

In [55]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

df_reduce_RFC_instances = datasets["reduce_mrmr_instances_hard"]
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Modelo base de XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# Definición del grid de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configuración de GridSearchCV con cv=2
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Entrenar GridSearchCV
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor puntaje
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejor puntuación de validación cruzada:", best_score)


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Mejores parámetros: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Mejor puntuación de validación cruzada: 0.9053672316384181


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:34:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [56]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_reduced)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[26525  1907]
 [    5    44]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.93      0.97     28432
Fraudulentas       0.02      0.90      0.04        49

    accuracy                           0.93     28481
   macro avg       0.51      0.92      0.50     28481
weighted avg       1.00      0.93      0.96     28481



# XGBoost con RFC instances

In [50]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

df_reduce_RFC_instances = datasets["reduce_RFC_instances"]
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Modelo base de XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# Definición del grid de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configuración de GridSearchCV con cv=2
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Entrenar GridSearchCV
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Obtener los mejores parámetros y el mejor puntaje
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejor puntuación de validación cruzada:", best_score)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Mejor puntuación de validación cruzada: 0.9237288135593221


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:53:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## Ejecutamos para el conjunto de prueba

In [51]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_reduce_RFC)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[28033   399]
 [    7    42]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      0.99     28432
Fraudulentas       0.10      0.86      0.17        49

    accuracy                           0.99     28481
   macro avg       0.55      0.92      0.58     28481
weighted avg       1.00      0.99      0.99     28481



# XGBoost con RFC hard instances

In [52]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

df_reduce_RFC_instances_hard = datasets["reduce_RFC_instances_hard"]
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Modelo base de XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# Definición del grid de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configuración de GridSearchCV con cv=2
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Entrenar GridSearchCV
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Mejores parámetros:", best_params)
print("Mejor puntuación de validación cruzada:", best_score)


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Mejores parámetros: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
Mejor puntuación de validación cruzada: 0.9237288135593221


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:53:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## Ejecutamos para el conjunto de prueba

In [54]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_reduce_RFC)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[27548   884]
 [    6    43]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.98     28432
Fraudulentas       0.05      0.88      0.09        49

    accuracy                           0.97     28481
   macro avg       0.52      0.92      0.54     28481
weighted avg       1.00      0.97      0.98     28481



# Comparacion

### **Tabla Comparativa de Modelos**

| Modelo                             | Precisión train | Precisión (Fraudes) | Recall (Fraudes) | Balance General                                       |
|------------------------------------|------------------|---------------------|------------------|------------------------------------------------------|
| **Gradient Boosting con RFC hard** | 0.51             | 0.06               | 0.88             | Buen recall, pero precisión baja para fraudes.       |
| **Gradient Boosting con mrMr**     | 0.53             | 0.02               | 0.92             | Alto recall, muchos falsos positivos.                |
| **Gradient Boosting con RFC soft** | 0.53            | 0.07               | 0.86             | Equilibrado, mejor que RFC hard en falsos positivos. |
| **XGBoost con mrMr hard**          | 0.51             | 0.02               | 0.90             | Prioriza recall, pero demasiados falsos positivos.   |
| **XGBoost con RFC instances**      | 0.51             | 0.10               | 0.86             | Mejor equilibrio entre precisión y recall.           |
| **XGBoost con RFC hard**           | 0.55             | 0.05               | 0.88             | Similar a Gradient Boosting RFC hard, menos preciso. |
